<h1>*Question.4 VaR</h1>
Bank has FX portfolio consisting of two currencies. Risk manager implemented historical VaR methodology as defined in internal policies (1 day with .99 confidence level) for the FX portfolio in excel, assuming no correlation between these currencies.
- implement the VaR calculation in python to revert one day VaR value

One year of historical data (market rate) for the currencies and current/spot portfolio value is known.


In [9]:
import pandas as pd
import numpy as np
from typing import List

class Shift:
    @staticmethod
    def logarithmic(rates: pd.Series, n: int) -> np.array:
        shifted_rates = rates.shift(n).dropna()
        division_vector = rates[n:] / shifted_rates
        ln_vector = np.log(division_vector) * np.sqrt(n)
        return np.exp(ln_vector) - 1
    
    @staticmethod
    def absolute():
        #TODO implement
        ...

    @staticmethod
    def relative():
        #TODO implement
        ...
    
class MarketRates:
    """
    Holds market rates time series for different products
    """
    def __init__(self, data: pd.DataFrame):
        self._data = data
    
    @classmethod
    def from_excel(cls, file_path: str):
        data = ( pd
                .read_excel(file_path)
                .set_index('date')
                .sort_index()
        )
        return cls(data)
    
    @property
    def data(self):
        return self._data
    
    
class Product:
    """
    Holds historical `market_rates` and current `spot_value` for a specific `product_name`.
    Calculates historical `pnl_vector` for the `product_name` for `n_periods` ahead.
    """
    def __init__(self, 
                 product_name: str, 
                 spot_value: float,
                 market_rates: pd.DataFrame, 
                 n_periods: int,
                 ):
        self._name = product_name
        self._hist_data = market_rates[product_name].copy()
        self._spot_value = spot_value
        self._n_periods = n_periods
    
    @property
    def name(self) -> str:
        return self._name
    
    @property
    def pnl_vector(self) -> np.ndarray:
        log_shift_vector = Shift.logarithmic(self._hist_data, self._n_periods)
        pnl_vector = self._spot_value * log_shift_vector
        return pnl_vector
    
class ValueAtRisk:
    """
    Calculates historical `total_pnl_vector` for multiple `products`
    assuming they are non-correlated. 
    Calculates Value-at-Risk based on given `weights` 
    for the least second and third elements of the `total_pnl_vector`.
    """
    def __init__(self, 
                 weights: List[float],
                 products: List[Product],
                 ):
        self._weights = np.array(weights)
        self._products = products
    
    @property
    def total_pnl_vector(self) -> np.array:
        pnl_vectors = [product.pnl_vector for product in self._products]
        return np.sum(pnl_vectors, axis=0)
    
    @property
    def value(self) -> float:
        total_pnl = self.total_pnl_vector
        worst_scenarios = [np.partition(total_pnl, n)[n] for n in (1,2)] #index starts with 0
        value = np.dot(self._weights, worst_scenarios)
        return value

In [10]:
market_rates = MarketRates.from_excel('inputs/market_rates.xlsx')
market_rates.data

,ccy-1,ccy-2
date,,
2018-11-14,1.149980,0.895255
2018-11-15,1.129599,0.905879
2018-11-16,1.126722,0.900414
2018-11-19,1.247645,0.910705
2018-11-20,1.123899,0.912825
...,...,...
2019-11-08,1.160726,0.877659
2019-11-11,1.166902,0.877539
2019-11-12,1.166031,0.883470


In [11]:
portfolio_values = {
    "ccy-1": 153084.81,
    "ccy-2": 95891.51
}

In [12]:
products = [Product(k, v, market_rates.data, 1) for k,v in portfolio_values.items()]

In [13]:
value_at_risk = ValueAtRisk([0.4, 0.6], products)

In [14]:
value_at_risk.value

-13572.733792468436